# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [132]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
### nltk.download('averaged_perceptron_tagger')
### nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## 0.1 Load and clean text data

In [133]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [134]:
## your code to subset to one press release and take the string
pharma = str(doj[doj["id"] == "17-1204"].contents)
pharma

'4909    The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.\xa0 We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.\xa0 And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RIC

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [135]:
## your code here to restrict to alpha
pharma_tokens = nltk.word_tokenize(pharma)
pharma_preprocess = [word for word in pharma_tokens if word.isalpha()]
pharma_preprocess

['The',
 'founder',
 'and',
 'majority',
 'owner',
 'of',
 'Insys',
 'Therapeutics',
 'was',
 'arrested',
 'today',
 'and',
 'charged',
 'with',
 'leading',
 'a',
 'nationwide',
 'conspiracy',
 'to',
 'profit',
 'by',
 'using',
 'bribes',
 'and',
 'fraud',
 'to',
 'cause',
 'the',
 'illegal',
 'distribution',
 'of',
 'a',
 'Fentanyl',
 'spray',
 'intended',
 'for',
 'cancer',
 'patients',
 'experiencing',
 'breakthrough',
 'pain',
 'More',
 'than',
 'Americans',
 'died',
 'of',
 'synthetic',
 'opioid',
 'overdoses',
 'last',
 'year',
 'and',
 'millions',
 'are',
 'addicted',
 'to',
 'opioids',
 'And',
 'yet',
 'some',
 'medical',
 'professionals',
 'would',
 'rather',
 'take',
 'advantage',
 'of',
 'the',
 'addicts',
 'than',
 'try',
 'to',
 'help',
 'them',
 'said',
 'Attorney',
 'General',
 'Jeff',
 'Sessions',
 'This',
 'Justice',
 'Department',
 'will',
 'not',
 'tolerate',
 'this',
 'We',
 'will',
 'hold',
 'accountable',
 'anyone',
 'from',
 'street',
 'dealers',
 'to',
 'corpora

In [136]:
## your code here for part of speech tagging
pos_tag_pharma = nltk.pos_tag(pharma_preprocess)
pos_tag_pharma

[('The', 'DT'),
 ('founder', 'NN'),
 ('and', 'CC'),
 ('majority', 'NN'),
 ('owner', 'NN'),
 ('of', 'IN'),
 ('Insys', 'NNP'),
 ('Therapeutics', 'NNP'),
 ('was', 'VBD'),
 ('arrested', 'VBN'),
 ('today', 'NN'),
 ('and', 'CC'),
 ('charged', 'VBN'),
 ('with', 'IN'),
 ('leading', 'VBG'),
 ('a', 'DT'),
 ('nationwide', 'JJ'),
 ('conspiracy', 'NN'),
 ('to', 'TO'),
 ('profit', 'VB'),
 ('by', 'IN'),
 ('using', 'VBG'),
 ('bribes', 'NNS'),
 ('and', 'CC'),
 ('fraud', 'NN'),
 ('to', 'TO'),
 ('cause', 'VB'),
 ('the', 'DT'),
 ('illegal', 'JJ'),
 ('distribution', 'NN'),
 ('of', 'IN'),
 ('a', 'DT'),
 ('Fentanyl', 'NNP'),
 ('spray', 'NN'),
 ('intended', 'VBD'),
 ('for', 'IN'),
 ('cancer', 'NN'),
 ('patients', 'NNS'),
 ('experiencing', 'VBG'),
 ('breakthrough', 'NN'),
 ('pain', 'NN'),
 ('More', 'JJR'),
 ('than', 'IN'),
 ('Americans', 'NNPS'),
 ('died', 'VBD'),
 ('of', 'IN'),
 ('synthetic', 'JJ'),
 ('opioid', 'NN'),
 ('overdoses', 'NNS'),
 ('last', 'JJ'),
 ('year', 'NN'),
 ('and', 'CC'),
 ('millions', 'NNS'

In [137]:
# sort adjectives 
adjectives = [one_tok[0] for one_tok in pos_tag_pharma 
              if one_tok[1] == "JJ" or 
              one_tok[1] == "JJR" or 
             one_tok[1] == "JJS"]

adj_sort = pd.DataFrame(adjectives).value_counts()[:5]
adj_sort

former        8
opioid        5
nationwide    4
other         3
addictive     3
Name: count, dtype: int64

## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [138]:
## your code here for part A
spacy_pharma = nlp(pharma)

entities = spacy_pharma.ents
entities

(4909,
 Insys Therapeutics Inc.,
 today,
 Fentanyl,
 More than 20,000,
 Americans,
 last year,
 millions,
 Jeff Sessions,
 This Justice Department,
 Trump,
 American,
 ”John N. Kapoor,
 74,
 Phoenix,
 Ariz.,
 the Board of Directors,
 Insys,
 this morning,
 Arizona,
 RICO,
 Kapoor,
 Executive,
 Board,
 Insys,
 Phoenix,
 today,
 U.S.,
 District Court,
 Boston,
 a later date,
 today,
 Boston,
 Insys,
 December 2016.The,
 Kapoor,
 Michael L. Babich,
 40,
 Scottsdale,
 Ariz.,
 Alec Burlakoff,
 42,
 Charlotte,
 N.C.,
 Richard M. Simon,
 46,
 Seal Beach,
 Calif.,
 Sunrise Lee,
 36,
 Bryant City,
 Mich.,
 Joseph A. Rowan,
 43,
 Panama City,
 Fla.,
 Managed Markets,
 Michael J. Gurry,
 53,
 Scottsdale,
 Ariz.,
 Subsys,
 Kapoor,
 six,
 Kapoor,
 United States,
 William D. Weinreb,
 Today,
 Insys,
 Harold H. Shaw,
 Charge of the Federal Bureau of Investigation,
 Boston Field Division,
 Today,
 FBI,
 Insys,
 Phillip Coyne,
 Charge for the Office of Inspector General of the U.S. Department of Health

In [139]:
## your code here for part B

pharma_LAW = [entity.text for entity in spacy_pharma.ents if (entity.label_ == "LAW")]

pharma_LAW

['RICO', 'the Controlled Substances Act', 'RICO']

C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

RICO is the Racketeer Influenced and Corrupt Organizations Act which makes enhanced penalties for crimes that are part of a criminal orgnization; this might apply to pharmaceutical kickbacks 


D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [140]:
## your code here
pattern = r"year|years"

possible_sentences = [entity.text for entity in spacy_pharma.ents if (entity.label_ == "DATE" and re.search(pattern, entity.text))]

possible_sentences



['last year', '20 years', 'three years', 'five years', 'three years']

E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [141]:
## your code here
pharma_phrases = nltk.sent_tokenize(pharma)

year_patterns = '|'.join(possible_sentences)

pharma_year_sentences = [sentence for sentence in pharma_phrases if re.search(year_patterns, sentence)]

pharma_year_sentences


['"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids.',
 'This investigation highlights our commitment to defending our mail system from illegal misuse and ensuring public trust in the mail.”“The U.S. Department of Veterans Affairs, Office of Inspector General will continue to aggressively investigate those that attempt to fraudulently impact programs designed to benefit our veterans and their families,” said Donna L. Neves, Special Agent in Charge of the VA OIG Northeast Field Office.The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.',
 'The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.']

The CEO will probably face at maximum 20 years in prison, 3 years of probation. 

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [142]:
## your code here for subsetting

doj_subset = doj[doj["topics_clean"].isin(["Civil Rights", "Hate Crimes", "Project Safe Childhood"])]
doj_subset.shape
doj_subset

(717, 6)

,id,title,contents,date,topics_clean,components_clean
77,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle"
155,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when they were asked to take temporary custody of a five-year-old child whose parents were deployed to Iraq with the U.S. Army. Evans sexually abused the child on multiple occasions during the 18 months that the child lived with him from May 2007 to December 2008. Trial Attorney Austin M. Berry of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Jacquelyn Hutzell of the Northern District of Alabama are prosecuting the case. U.S. Army Criminal Investigations Division and the FBI’s Birmingham, Alabama, Division investigated the case. This case was brought as part of Project Safe Childhood, a nationwide initiative to combat the growing epidemic of child sexual exploitation and abuse, launched in May 2006 by the Department of Justice. Led by U.S. Attorneys’ offices and CEOS, Project Safe Childhood marshals federal, state and local resources to better locate, apprehend and prosecute individuals who exploit children via the Internet, as well as to identify and rescue victims. For more information about Project Safe Childhood, please visit www.justice.gov/psc.",2015-12-11T00:00:00-05:00,Project Safe Childh

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [143]:
## your code here to define function
def sentiment_analysis(press_release): 
    
    tokens = nltk.word_tokenize(press_release)
    preprocess = ' '.join(word for word in tokens if word.isalpha())
    
    spacy_release = nlp(preprocess)
    entities = [entity.text for entity in spacy_release.ents]


    pattern = '|'.join(entities)
    no_entities = re.sub(pattern, "", press_release)


    sent_obj = SentimentIntensityAnalyzer()
    sentiment = sent_obj.polarity_scores(no_entities)


    return sentiment 





In [144]:
## your code here executing the function
#apply function to doj_subset
sentiments = []
for press_release in doj_subset.contents: 
    sentiments.append(sentiment_analysis(press_release))

sentiments

[{'neg': 0.194, 'neu': 0.758, 'pos': 0.048, 'compound': -0.9931},
 {'neg': 0.115, 'neu': 0.811, 'pos': 0.074, 'compound': -0.891},
 {'neg': 0.083, 'neu': 0.84, 'pos': 0.077, 'compound': -0.4767},
 {'neg': 0.112, 'neu': 0.801, 'pos': 0.086, 'compound': -0.8305},
 {'neg': 0.168, 'neu': 0.79, 'pos': 0.042, 'compound': -0.9864},
 {'neg': 0.137, 'neu': 0.814, 'pos': 0.049, 'compound': -0.987},
 {'neg': 0.14, 'neu': 0.779, 'pos': 0.081, 'compound': -0.9413},
 {'neg': 0.073, 'neu': 0.856, 'pos': 0.07, 'compound': -0.25},
 {'neg': 0.096, 'neu': 0.846, 'pos': 0.058, 'compound': -0.9022},
 {'neg': 0.145, 'neu': 0.793, 'pos': 0.061, 'compound': -0.9741},
 {'neg': 0.206, 'neu': 0.759, 'pos': 0.034, 'compound': -0.9973},
 {'neg': 0.094, 'neu': 0.84, 'pos': 0.066, 'compound': -0.8555},
 {'neg': 0.074, 'neu': 0.858, 'pos': 0.068, 'compound': -0.2263},
 {'neg': 0.291, 'neu': 0.681, 'pos': 0.029, 'compound': -0.9951},
 {'neg': 0.17, 'neu': 0.775, 'pos': 0.055, 'compound': -0.9907},
 {'neg': 0.125, 'neu

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [145]:
## your code here

doj_subset_wscore = doj_subset.copy()
doj_subset_wscore['neg'] = [sentiment['neg'] for sentiment in sentiments]
doj_subset_wscore['pos'] = [sentiment['pos'] for sentiment in sentiments]
doj_subset_wscore['neu'] = [sentiment['neu'] for sentiment in sentiments]
doj_subset_wscore['compound'] = [sentiment['compound'] for sentiment in sentiments]

doj_subset_neg_sort = doj_subset_wscore.sort_values(by='neg', ascending=False)
doj_subset_neg_sort[["id", "contents", "neg"]][:2]


,id,contents,neg
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",0.298
329,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.291


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [164]:
## agg and find the mean compound score by topic
doj_subset_wscore.groupby('topics_clean').agg({'compound': 'mean'})

,compound
topics_clean,
Civil Rights,-0.088586
Hate Crimes,-0.932882
Project Safe Childhood,-0.530237


Hate crimes might be more negative because they are more violent and emotionally charged, whereas civil rights might be a more encompassing category that is not necearily always violent.

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [52]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [147]:
## your code defining a text processing function

stop_words = stopwords.words('english')
custom_words = custom_doj_stopwords + stop_words
                            

snow_stemmer = SnowballStemmer(language='english') 

def text_processing(string): 
    str_lower = string.lower()
    tokens = word_tokenize(str_lower)  # Tokenize the string into words
    stems = [snow_stemmer.stem(token) 
             for token in tokens 
             if token not in custom_words
             and token.isalpha() 
             and len(token) >= 4]  # Filter out stopwords and non-alpha words, and stem

    final_string = ' '.join(stems)
    return final_string
    

In [148]:
## your code executing the function
doj_subset_wscore["processed_text"] = doj_subset_wscore["contents"].apply(text_processing)
doj_subset_wscore

,id,title,contents,date,topics_clean,components_clean,neg,pos,neu,compound,processed_text
77,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle",0.194,0.048,0.758,-0.9931,former supervisori correct offic louisiana state penitentiari angola louisiana plead guilti yesterday connect beat handcuf shackl inmat addit conspir cover misconduct falsifi offici record lie intern investig happen jame savoy marksvill louisiana admit plea hear wit offic use excess forc inmat fail interven conspir offic cover beat engag varieti obstruct act person falsifi offici prison record cover attack scotti kennedi beeb arkansa john sander marksvill louisiana previous plead guilti novemb septemb role beat cover everi citizen right process protect unreason forc correct offic violat basic constitut must held account egregi action said act general john gore continu vigor prosecut correct offic violat public trust commit crime cover violat feder crimin yesterday anoth exampl unwav commit pursu violat feder crimin law said act unit state middl louisiana corey amundson continu work close ensur investig baton roug resid agenc prosecut frederick menner middl louisiana christoph perra crimin section
155,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when they were asked to take temporary custody of a five-year-old child whose parents were de

In [149]:
## your code showing the examples
doj_subset_wscore[doj_subset_wscore["id"] == "16-718"]
doj_subset_wscore[doj_subset_wscore["id"] == "16-217"]

,id,title,contents,date,topics_clean,components_clean,neg,pos,neu,compound,processed_text
11593,16-718,Three Mississippi Correctional Officers Indicted for Inmate Assault and Cover-Up,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment",2016-06-21T00:00:00-04:00,Civil Rights,"Civil Rights Division; Civil Rights - Criminal Section; USAO - Mississippi, Northern",0.298,0.032,0.67,-0.9968,indict unseal today mississippi correct offic charg beat inmat third charg help cover indict charg lawardrick marsher robert sturdiv offic mississippi state penitentiari parchman mississippi beat includ kick punch throw victim ground marsher sturdiv charg violat right convict prison free cruel unusu punish sturdiv also charg fail interven marsher punch beat indict alleg action involv danger weapon result bodili injuri victim third offic deont pate charg along marsher sturdiv conspir cover beat indict alleg three offic submit fals report three lie convict marsher sturdiv face maximum sentenc year prison excess forc charg three offic face five year prison conspiraci fals statement charg year prison fals report charg indict mere accus defend presum innoc unless proven guilti investig jackson cooper mississippi correct prosecut robert coleman northern mississippi dana mulhaus crimin section marsher indict


,id,title,contents,date,topics_clean,components_clean,neg,pos,neu,compound,processed_text
6727,16-217,Justice Department Reaches Agreement with the City of Miami and the Miami Police Department to Implement Reforms on Officer-Involved Shootings,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-involved shootings and to more effectively and quickly investigate officer-involved shootings that do occur, through measures that include: “This settlement represents a renewed commitment by the city of Miami and Chief Rodolfo Llanes to provide constitutional policing for Miami residents and to protect public safety through sustainable reform,” said Principal Deputy Assistant Attorney General Gupta. “The agreement will help to strengthen the relationship between the MPD and the communities they serve by improving accountability for officers who fire their weapons unlawfully, and provides for community participation in the enforcement of this agreement.” “Today's agreement is the result of a joint effort between the Department of Justice and the City of Miami to ensure that the Miami Police Department continues its efforts to make our community safe while protecting the sacred Constitutional rights of all of our citizens,” said U.S. Attorney Ferrer. “Through oversight and communication, the agreement seeks to make permanent the positive changes that former Chief Orosa and Chief Llanes have made, and we applaud the City Commission’s vote.” The settlement agreement builds upon important reforms implemented by the city since the Justice Department issued its findings, including: The investigation was conducted by attorneys and staff from the Civil Rights Division’s Special Litigation Section and the Civil Division of the U. S. Attorney’s Office of the Southern District of Florida.",2016-02-25T00:00:00-05:00,Civil Rights,"Civil Rights Division; Civil Rights - Special Litigation Section; USAO - Florida, Southern",0.026,0.216,0.758,0.9967,reach comprehens settlement agreement citi miami miami polic resolv shoot offic announc princip deputi general vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim shoot effect quick investig shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [150]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [151]:
# your code here

dtm = create_dtm(doj_subset_wscore.processed_text, 
           metadata = doj_subset_wscore[["id", "compound", "topics_clean"]]
          )
dtm

,index,id,compound,topics_clean,aaron,abandon,abbat,abbi,abbott,abdomen,...,zamora,zane,zealand,zealous,zeeman,zero,zobel,zone,zunggeemog,zwengel
0,77,17-1235,-0.9931,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,155,15-1522,-0.8910,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,157,16-213,-0.4767,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,162,16-381,-0.8305,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,168,14-464,-0.9864,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,13002,09-368,-0.9664,Hate Crimes,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
713,13032,18-775,0.8481,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
714,13034,12-596,-0.9313,Project Safe Childhood,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
715,13068,18-359,-0.9723,Civil Rights,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [152]:
in_top_5_percentile = doj_subset_wscore['compound'].quantile(0.95) 
in_bottom_5_percentile = doj_subset_wscore['compound'].quantile(0.05) 

top_sentiment = doj_subset_wscore[doj_subset_wscore['compound'] >= in_top_5_percentile]["id"]
bottom_sentiment = doj_subset_wscore[doj_subset_wscore['compound'] <= in_bottom_5_percentile]["id"]

dtm_top = dtm[dtm["id"].isin(top_sentiment)]
dtm_bottom = dtm[dtm["id"].isin(bottom_sentiment)]

top_word_frequencies = dtm_top[dtm_top.columns[4:]].sum(axis=0)
bottom_word_frequencies = dtm_bottom[dtm_bottom.columns[4:]].sum(axis=0)

top_word_frequencies.sort_values(ascending=False).head(10)
bottom_word_frequencies.sort_values(ascending=False).head(10)



agreement     173.0
enforc        134.0
state         116.0
ensur         111.0
disabl        102.0
communiti     100.0
polic          93.0
settlement     86.0
general        86.0
student        85.0
dtype: float64

assault     191.0
victim      187.0
offic       172.0
crime       170.0
defend      131.0
sentenc     128.0
hate        123.0
feder       118.0
charg       113.0
prosecut    107.0
dtype: float64

In [153]:
def get_topwords(subset, top_n=10):
    word_frequencies = subset[subset.columns[4:]].sum(axis=0)
    return word_frequencies.sort_values(ascending=False).head(top_n)

# Top 10 words for press releases with compound sentiment in the top 5%
top_10_top_sentiment = get_topwords(dtm_top)
print("Top 10 words for top 5% sentiment:\n", top_10_top_sentiment)

# Top 10 words for press releases with compound sentiment in the bottom 5%
top_10_bottom_sentiment = get_topwords(dtm_bottom)
print("Top 10 words for bottom 5% sentiment:\n", top_10_bottom_sentiment)

# Top 10 words for each topic
unique_topics = dtm['topics_clean'].unique()
for topic in unique_topics:
    topic_subset = dtm[dtm['topics_clean'] == topic]
    top_words = get_topwords(topic_subset)
    print(f"Top 10 words for topic '{topic}':\n", top_words)


Top 10 words for top 5% sentiment:
 agreement     173.0
enforc        134.0
state         116.0
ensur         111.0
disabl        102.0
communiti     100.0
polic          93.0
settlement     86.0
general        86.0
student        85.0
dtype: float64
Top 10 words for bottom 5% sentiment:
 assault     191.0
victim      187.0
offic       172.0
crime       170.0
defend      131.0
sentenc     128.0
hate        123.0
feder       118.0
charg       113.0
prosecut    107.0
dtype: float64
Top 10 words for topic 'Civil Rights':
 offic        627.0
hous         620.0
discrimin    541.0
enforc       531.0
disabl       509.0
said         497.0
feder        475.0
violat       470.0
state        443.0
general      408.0
dtype: float64
Top 10 words for topic 'Project Safe Childhood':
 child          1018.0
exploit         698.0
sexual          570.0
safe            476.0
childhood       472.0
project         472.0
pornographi     447.0
children        416.0
crimin          404.0
prosecut        374.0


## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [165]:

text_raw_tokens = [wordpunct_tokenize(one_text) for one_text in doj_subset_wscore.processed_text]
text_raw_dict = corpora.Dictionary(text_raw_tokens)


corpus_fromdict = [text_raw_dict.doc2bow(one_text) 
                   for one_text in text_raw_tokens]

ldamod = gensim.models.ldamodel.LdaModel(corpus_fromdict, 
                                         num_topics = 3, 
                                         id2word=text_raw_dict, 
                                         passes=6, 
                                         alpha = 'auto',
                                         per_word_topics = True, 
                                         random_state = 91988)

        
topics = ldamod.print_topics(num_words=15)

for topic in topics:
    print(topic)

(0, '0.015*"child" + 0.013*"victim" + 0.011*"sentenc" + 0.011*"prosecut" + 0.010*"exploit" + 0.010*"sexual" + 0.008*"guilti" + 0.008*"feder" + 0.008*"crimin" + 0.007*"investig" + 0.007*"charg" + 0.007*"prison" + 0.007*"safe" + 0.007*"project" + 0.007*"childhood"')
(1, '0.009*"offic" + 0.008*"enforc" + 0.008*"disabl" + 0.007*"agreement" + 0.006*"said" + 0.006*"feder" + 0.006*"state" + 0.006*"violat" + 0.006*"court" + 0.005*"polic" + 0.005*"general" + 0.005*"today" + 0.005*"provid" + 0.005*"ensur" + 0.005*"counti"')
(2, '0.013*"hous" + 0.009*"discrimin" + 0.009*"said" + 0.007*"feder" + 0.007*"alleg" + 0.007*"charg" + 0.006*"defend" + 0.006*"indict" + 0.005*"today" + 0.005*"violat" + 0.005*"general" + 0.005*"state" + 0.005*"cross" + 0.005*"nation" + 0.005*"prosecut"')


## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [166]:
## your code here to get doc-level topic probabilities 

document_topic_probs = [ldamod.get_document_topics(item, minimum_probability=0) for item in corpus_fromdict]
assert len(document_topic_probs) == len(doj_subset_wscore), "Length of document-level topic probabilities list does not match the number of rows in doj_subset_wscore"

In [174]:
## your code here to add those topic probabilities to the dataframe

n_topics = 3

topic_probs_bydoc = [ldamod.get_document_topics(item) for item in corpus_fromdict]

one_list_tup = topic_probs_bydoc[0]

topic_probs_bydoc_long = pd.DataFrame([t for lst in topic_probs_bydoc for t in lst],
                                     columns=['topic', 'probability'])

# Create a placeholder 'doc_id' column with the same length as the DataFrame
topic_probs_bydoc_long['doc_id'] = range(len(topic_probs_bydoc_long))

topic_probs_bydoc_wide = pd.pivot_table(topic_probs_bydoc_long, index=['doc_id'],
                                        columns=['topic'], values='probability').reset_index()

topic_probs_bydoc_wide.columns = ['doc_id'] + ["topic_" + str(i) for i in np.arange(0, n_topics)]

topic_probs_bydoc_wide

topic_wmeta = pd.merge(topic_probs_bydoc_wide,
                       doj_subset_wscore,
                       left_on='doc_id',
                       right_on='id', 
                      right_index=True)


topic_wmeta['toptopic'] = topic_wmeta[[col for col in topic_wmeta.columns if 
                                       "topic_" in col]].idxmax(axis=1)

topic_wmeta


,doc_id,topic_0,topic_1,topic_2
0,0,0.162811,NaN,NaN
1,1,NaN,0.554110,NaN
2,2,NaN,NaN,0.283079
3,3,0.998722,NaN,NaN
4,4,0.999250,NaN,NaN
...,...,...,...,...
1059,1059,0.999147,NaN,NaN
1060,1060,0.101984,NaN,NaN
1061,1061,NaN,0.171173,NaN
1062,1062,NaN,NaN,0.726843


,doc_id,topic_0,topic_1,topic_2,id,title,contents,date,topics_clean,components_clean,neg,pos,neu,compound,processed_text,toptopic
77,77,0.999223,NaN,NaN,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle",0.194,0.048,0.758,-0.9931,former supervisori correct offic louisiana state penitentiari angola louisiana plead guilti yesterday connect beat handcuf shackl inmat addit conspir cover misconduct falsifi offici record lie intern investig happen jame savoy marksvill louisiana admit plea hear wit offic use excess forc inmat fail interven conspir offic cover beat engag varieti obstruct act person falsifi offici prison record cover attack scotti kennedi beeb arkansa john sander marksvill louisiana previous plead guilti novemb septemb role beat cover everi citizen right process protect unreason forc correct offic violat basic constitut must held account egregi action said act general john gore continu vigor prosecut correct offic violat public trust commit crime cover violat feder crimin yesterday anoth exampl unwav commit pursu violat feder crimin law said act unit state middl louisiana corey amundson continu work close ensur investig baton roug resid agenc prosecut frederick menner middl louisiana christoph perra crimin section,topic_0
155,155,NaN,NaN,0.334376,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when

In [175]:
## your code here to summarize the topic proportions for each of the topics_clean 
cross_tab = pd.crosstab(index = topic_wmeta['topics_clean'], columns = topic_wmeta['toptopic'], normalize="index")

cross_tab

toptopic,topic_0,topic_1,topic_2
topics_clean,,,
Civil Rights,0.428571,0.142857,0.428571
Hate Crimes,0.280000,0.440000,0.280000
Project Safe Childhood,0.454545,0.090909,0.454545


In [110]:
# code for example
topic_wmeta[topic_wmeta["topics_clean"] == "Civil Rights"].sample(n=1).contents
topic_wmeta[topic_wmeta["topics_clean"] == "Hate Crimes"].sample(n=1).contents
topic_wmeta[topic_wmeta["topics_clean"] == "Project Safe Childhood"].sample(n=1).contents

851    Attorney General Jeff Sessions issued the following proclamation commemorating January as National Slavery and Human Trafficking Prevention Month: “Human trafficking is a nationwide public health and civil rights crisis. Its victims are everywhere: at truck stops, in cities, in rural areas, and in suburbs, and who now total an unconscionable 25 million victims globally according to some estimates. That means 25 million human beings—parents, siblings, and children—have been coerced into a commercial sex act, forced into labor, or exploited because they desperately seek a better life. It is a priority of the Department of Justice to combat this depraved and predatory behavior through swift and aggressive enforcement of our nation’s laws to bring traffickers to justice and restore the lives of victims and survivors. “The Justice Department’s U.S. Attorneys’ Offices, working closely with the Federal Bureau of Investigation (FBI), other federal agencies, and our state, local, and tri

496    WASHINGTON - Gary Dodson, 32, of Waldron, Ark., pleaded guilty today in U.S. District Court in Little Rock, Ark., to one count of civil rights conspiracy, one count of interference with housing rights due to race and one count of possession of an unregistered firearm/destructive device for his involvement in the Jan. 14, 2011, racially motivated firebombing of the home of an interracial couple in Hardy, Ark. Dodson, along with Jason Barnwell, 37, of Evening Shade, Ark.; Jake Murphy, 19, of Waldron; Dustin Hammond, 20, of Hardy, Ark.; and Wendy Treybig, 31, of Evening Shade, were indicted in April by a federal grand jury on civil rights charges and other federal charges stemming from their participation in the racially motivated firebombing and their attempts to obstruct a federal investigation. During the plea proceedings, Dodson admitted that on the night of Jan. 14, 2011, while at a party at Barnwell’s house in Evening Shade, he, Murphy, Hammond and Barnwell devised a plan to 

157    An Alabama native was indicted today and charged with multiple crimes involving travel with intent to engage in illicit sexual conduct with minors and child pornography, announced Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Kenyen R. Brown of the Southern District of Alabama. Clarence Edward Evers Jr., aka Bud, a technology teacher employed by the Conecuh County, Alabama, Board of Education, was arrested on Feb. 11, 2016, and was charged today with five counts of travel with intent to engage in illicit sexual conduct with a minor, one count of attempted travel with intent to engage in illicit sexual conduct with a minor, one count of production and attempted production of child pornography, one count of transportation of child pornography, one count of receipt of child pornography, one count of access with intent to view child pornography and one count of possession of child pornography. According to the indictmen

Some manual topics may map more cleanly to an estimated topic than others because their contents span a wider range of categories. For example, trafficking is involved in both the civil rights and Project Safe Childhood sample releases, and this overlap might make mapping difficult. 

# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [111]:
## your code here 

def create_bigram_onedoc(string): 
    bigrams = []
    

    words = nltk.word_tokenize(string)
    
    # Iterate over the words, combining consecutive pairs into bigrams
    for i in range(len(words) - 1):
        bigram = words[i] + '_' + words[i+1]
        bigrams.append(bigram)

    final_bigrams = ' '.join(bigrams)

    return final_bigrams

doj_subset_wscore['processed_text_bigrams'] = doj_subset_wscore['processed_text'].apply(create_bigram_onedoc)

doj_subset_wscore[doj_subset_wscore["id"] == "16-217"][["id", "processed_text", "processed_text_bigrams"]]

,id,processed_text,processed_text_bigrams
6727,16-217,reach comprehens settlement agreement with citi miami miami polic resolv shoot offic announc princip deputi general vanita gupta head wifredo ferrer southern florida settlement which approv miami citi commiss today will into effect when agreement sign parti resolv claim stem from into shoot offic which conduct under violent crime control enforc find issu juli identifi pattern practic excess forc through shoot violat fourth amend constitut citi complianc with settlement will monitor independ review former tampa florida polic chief jane castor under settlement agreement citi will implement comprehens reform ensur constitut polic support public trust settlement agreement design minim shoot more effect quick investig shoot that occur through measur that includ this settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti through sustain reform said princip deputi general gupta agreement will help strengthen relationship between communiti they serv improv account offic fire their weapon unlaw provid communiti particip enforc this today agreement result joint effort between citi miami ensur that miami polic continu effort make communiti safe while protect sacr constitut citizen said ferrer through oversight communic agreement seek make perman posit chang that former chief orosa chief llane have made applaud citi commiss settlement agreement build upon import reform implement citi sinc issu find includ conduct attorney staff from special litig section southern florida,reach_comprehens comprehens_settlement settlement_agreement agreement_with with_citi citi_miami miami_miami miami_polic polic_resolv resolv_shoot shoot_offic offic_announc announc_princip princip_deputi deputi_general general_vanita vanita_gupta gupta_head head_wifredo wifredo_ferrer ferrer_southern southern_florida florida_settlement settlement_which which_approv approv_miami miami_citi citi_commiss commiss_today today_will will_into into_effect effect_when when_agreement agreement_sign sign_parti parti_resolv resolv_claim claim_stem stem_from from_into into_shoot shoot_offic offic_which which_conduct conduct_under under_violent violent_crime crime_control control_enforc enforc_find find_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_through through_shoot shoot_violat violat_fourth fourth_amend amend_constitut constitut_citi citi_complianc complianc_with with_settlement settlement_will will_monitor monitor_independ independ_review review_former former_tampa tampa_florida florida_polic polic_chief chief_jane jane_castor castor_under under_settlement settlement_agreement agreement_citi citi_will will_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_trust trust_settlement settlement_agreement agreement_design design_minim minim_shoot shoot_more more_effect effect_quick quick_investig investig_shoot shoot_that that_occur occur_through through_measur measur_that that_includ includ_this this_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_through through_sustain sustain_reform reform_said said_princip princip_deputi deputi_general general_gupta gupta_agreement agreement_will will_help help_strengthen strengthen_relationship relationship_between between_communiti communiti_they they_serv serv_improv improv_account account_offic offic_fire fire_their their_weapon weapon_unlaw unlaw_provid provid_communiti communiti_particip particip_enforc enforc_this this_today today_agreement agreement_result result_joint joint_effort effort_between between_citi citi_miami miami_ensur ensur_that that_miami miami_polic polic_continu continu_effort effort_make make_co

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [112]:
# your code here
dtm_bigram = create_dtm(
    list_of_strings=doj_subset_wscore['processed_text_bigrams'],
    metadata=doj_subset_wscore[['id', 'compound', 'topics_clean']]
)
dtm_shape = dtm.shape

dtm_bigram_shape = dtm_bigram.shape


print(f"Unigram DTM dimensions: {dtm_shape}")
print(f"Bigram DTM dimensions: {dtm_bigram_shape}")
print("The bigram DTM has more dimensions than the unigram DTM because each bigram represents a unique word pair, while each unigram represents a single word. The combinations of bigrams exceed the total number of unique unigrams, resulting in more columns (features) in the bigram DTM.")


for topic in dtm_bigram['topics_clean'].unique():
    topic_subset = dtm_bigram[dtm_bigram['topics_clean'] == topic]
    top_bigrams = get_topwords(topic_subset)
    print(f"Top 10 bigrams for topic '{topic}':\n", top_bigrams)


Unigram DTM dimensions: (717, 6819)
Bigram DTM dimensions: (717, 76265)
The bigram DTM has more dimensions than the unigram DTM because each bigram represents a unique word pair, while each unigram represents a single word. The combinations of bigrams exceed the total number of unique unigrams, resulting in more columns (features) in the bigram DTM.
Top 10 bigrams for topic 'Civil Rights':
 with_disabl       320
alleg_that        276
fair_hous         227
deputi_general    221
princip_deputi    221
ensur_that        211
vanita_gupta      202
gupta_head        200
general_vanita    199
said_princip      186
dtype: int64
Top 10 bigrams for topic 'Project Safe Childhood':
 project_safe         472
safe_childhood       472
child_pornographi    445
child_exploit        279
sexual_exploit       223
exploit_children     198
plead_guilti         197
exploit_obscen       175
child_sexual         174
obscen_section       174
dtype: int64
Top 10 bigrams for topic 'Hate Crimes':
 hate_crime       

dtm_bigram has more rows than the first dtm because there are more unique combinations of bigrams than there are unnique words. 

# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [114]:
# your code here 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(doj_subset_wscore['processed_text'])

pairwise_similarity = cosine_similarity(tfidf_matrix, tfidf_matrix)

# makes it so won't find similarity with self 
np.fill_diagonal(pairwise_similarity, -1)

top_pairs_indices = np.unravel_index(pairwise_similarity.argmax(), pairwise_similarity.shape)
top_pairs_similarity = pairwise_similarity[top_pairs_indices]

doc_id1 = doj_subset_wscore.iloc[top_pairs_indices[0]]['id']
doc_id2 = doj_subset_wscore.iloc[top_pairs_indices[1]]['id']

doc_id1
doc_id2

top_pair1_text = doj_subset_wscore[doj_subset_wscore['id'] == doc_id1]['processed_text'].iloc[0]
top_pair2_text = doj_subset_wscore[doj_subset_wscore['id'] == doc_id2]['processed_text'].iloc[0]

top_pair1_text
top_pair2_text


'17-546'

'17-231'

'church hill maryland resid sentenc today year prison follow lifetim term supervis releas entic minor engag sexual activ attempt transfer obscen materi minor announc act general kenneth blanco crimin act benjamin greenberg southern florida robert moor plead guilti march befor judg daniel hurley southern florida moor employ secret assign white hous time arrest remain custodi sinc that time moor sinc been termin from secret servic posit accord admiss made connect with plea moor maintain profil social media applic which provid platform exchang digit imag well voic text messag delawar state polic detect with delawar child predat task forc creat profil this site pose girl with whom moor engag number onlin chat session mobil app over period includ while moor work number onlin chat between moor undercov offic pose femal minor were sexual natur sever occas moor sent pictur himself includ sexual explicit imag accord plea document after arrest enforc discov that moor communic with minor florida 

'church hill maryland resid plead guilti today feder court count entic minor engag sexual activ count attempt transfer obscen materi minor announc act general kenneth blanco crimin wifredo ferrer southern florida robert moor plead guilti today befor judg daniel hurley southern florida moor employ secret assign white hous time arrest remain custodi sinc that time moor sinc been termin from secret servic posit accord admiss made connect with plea moor maintain profil social media applic which provid platform exchang digit imag well voic text messag delawar state polic detect with delawar child predat task forc creat profil this site pose girl with whom moor engag number onlin chat session mobil app over period includ while moor work number onlin chat between moor undercov offic pose femal minor were sexual natur sever occas moor sent pictur himself includ sexual explicit imag accord plea document after arrest enforc discov that moor communic with minor florida moor admit that those commu

These two press releases appear to be about the same case. They are both in Church Hill Maryland involving Kenneth Blanco and sending sexual pictures to minors. ID 17-546 is the sentencing press release and ID 17-231 is the guilty plea release. 